In [1]:
import os
import torch
import random
import datetime
import pandas as pd
import numpy as np

from torch.utils.data import Dataset
from src.datasets import RL4RS, ContentWise, DummyData
from src.utils import train, get_dummy_data, get_train_val_test_tmatrix_tnumitems, get_svd_encoder
from src.embeddings import RecsysEmbedding

experiment_name = 'FlattenedGRU'
device = 'cuda:1'
seed = 123
pkl_path = '../data/'

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [2]:
torch.__version__

'1.9.0'

# Модель

In [3]:
import torch.nn.functional as F
torch.autograd.set_detect_anomaly(True)

class NeuralClickModel(torch.nn.Module):
    def __init__(self, embedding, output_dim=1, dropout = 0.1):
        super().__init__()
        self.embedding_dim = embedding.embedding_dim
        self.embedding = embedding
        self.rnn_layer = torch.nn.GRU(
            input_size = embedding.embedding_dim, 
            hidden_size = embedding.embedding_dim, 
            batch_first = True,
            dropout=dropout
        )
        self.out_layer = torch.nn.Linear(embedding.embedding_dim, output_dim)


    def forward(self, batch):
        shp = batch['slates_item_indexes'].shape
        item_embs, user_embs = self.embedding(batch)
        item_embs = item_embs.flatten(-3, -2)
        
        # while training, let out model see the future 
        # while testing, it can see only the 
        if self.training:
            indices = (batch['length'] - 1)
        else:
            indices = (batch['in_length'] - 1)
        
        indices[indices<0] = 0
        indices = indices[:, None, None].repeat(1, 1, user_embs.size(-1))
        user_embs = user_embs.gather(1, indices).squeeze(-2).unsqueeze(0)

#         print(indices.shape, user_embs.shape, item_embs.shape, )
        rnn_out, _ = self.rnn_layer(
            item_embs,
            user_embs,
        )
        return self.out_layer(rnn_out).reshape(shp)

d = DummyData()
dummy_loader, dummy_matrix = get_dummy_data(d)
emb = RecsysEmbedding(d.n_items, dummy_matrix, embeddings='explicit')

for batch in dummy_loader:
    break
    
model = NeuralClickModel(
    RecsysEmbedding(d.n_items, dummy_matrix, embeddings='neural'),
    output_dim=1
).to('cpu')

model(batch)


3it [00:00, 3416.48it/s]

biulding affinity matrix...



/opt/conda/lib/python3.7/site-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


tensor([[[-0.1939, -0.2007,  0.0153],
         [ 0.1096, -0.0981, -0.0877]],

        [[-0.0586, -0.0425, -0.1067],
         [-0.1452, -0.1602, -0.1639]]], grad_fn=<ViewBackward>)

# Игрушечный датасет: проверим, что сходится к идеальным метрикам

In [4]:
d = DummyData()
dummy_loader, dummy_matrix = get_dummy_data(d)

for embeddings in ['explicit', 'svd', 'neural']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
    model = NeuralClickModel(
        RecsysEmbedding(d.n_items, dummy_matrix, embeddings='neural').to('cpu'),
        output_dim=1
    ).to('cpu')
    _, metrics = train(model, 
       dummy_loader, dummy_loader, dummy_loader, device=device, lr=1e-3, num_epochs=5000, dummy=True,
       silent=True,
#        debug=True,
    
    )


3it [00:00, 3804.93it/s]

biulding affinity matrix...

Evaluating FlattenedGRU with explicit embeddings



train... loss:0.6934366822242737:   0%|                                                                                      | 0/5000 [00:00<?, ?it/s]

Test before learning: {'f1': 0.0, 'roc-auc': 0.6666666269302368, 'accuracy': 0.5}


train... loss:0.6817811131477356:   0%|                                                                              | 2/5000 [00:00<21:20,  3.90it/s]

...

train... loss:0.6130875945091248:   0%|▏                                                                             | 9/5000 [00:01<17:05,  4.87it/s]

Test update: epoch: 7 |accuracy: 0.75 | f1: 0.0 | auc: 1.0 | treshold: 0.52


train:   0%|                                                                                                                 | 0/5000 [00:00<?, ?it/s]

Test update: epoch: 9 |accuracy: 1.0 | f1: 1.0 | auc: 1.0 | treshold: 0.53

Evaluating FlattenedGRU with neural embeddings
Test before learning: {'f1': 1.0, 'roc-auc': 1.0, 'accuracy': 1.0}


train... loss:0.645214855670929:   0%|                                                                                       | 0/5000 [00:00<?, ?it/s]


# ContentWise

In [5]:
# c = ContentWise('../data/CW/data/ContentWiseImpressions/CW10M-CSV/')
# for item in tqdm(c):
#     pass
# c.dump(os.path.join(pkl_path, 'cw.pkl'))

In [6]:
content_wise_results = []
c = ContentWise.load(os.path.join(pkl_path, 'cw.pkl'))
c_train_loader, c_val_loader, c_test_loader, c_train_user_item_matrix, train_num_items = get_train_val_test_tmatrix_tnumitems(c, batch_size=150)
len(c_train_loader), len(c)

(108, 20216)

In [7]:
for embeddings in ['svd', 'neural']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
    model = NeuralClickModel(
        RecsysEmbedding(train_num_items, c_train_user_item_matrix, embeddings='neural').to('cpu'),
        output_dim=1
    ).to(device)

    _, metrics = train(model, 
       c_train_loader, c_val_loader, c_test_loader, device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
       silent=True, 
    )
    
    metrics['embeddings'] = embeddings
    content_wise_results.append(metrics)
    
pd.DataFrame(content_wise_results).to_csv(f'results/cw_{experiment_name}.csv')


Evaluating FlattenedGRU with svd embeddings


train:   0%|                                                                                                                 | 0/5000 [00:00<?, ?it/s]

Test before learning: {'f1': 0.13663055002689362, 'roc-auc': 0.4311465620994568, 'accuracy': 0.4371917247772217}


train... loss:43.564214169979095:   0%|                                                                           | 1/5000 [00:31<44:03:51, 31.73s/it]

Test update: epoch: 0 |accuracy: 0.09900882095098495 | f1: 0.18017838895320892 | auc: 0.6250038146972656 | treshold: 0.05


...

train... loss:29.420453891158104:   0%|▎                                                                         | 19/5000 [06:09<26:52:41, 19.43s/it]

Test update: epoch: 18 |accuracy: 0.7316352128982544 | f1: 0.29893094301223755 | auc: 0.7338671088218689 | treshold: 0.16


train... loss:28.49056200683117:   1%|▍                                                                          | 29/5000 [09:18<26:22:45, 19.10s/it]

Test update: epoch: 28 |accuracy: 0.7677657008171082 | f1: 0.30687782168388367 | auc: 0.734139084815979 | treshold: 0.17


train... loss:27.23810836672783:   1%|▉                                                                          | 65/5000 [20:49<26:20:54, 19.22s/it]


# RL4RS

In [8]:
# r = RL4RS('../data/rl4rs-dataset/')
# for item in tqdm(r):
#     pass
# r.dump(os.path.join(pkl_path, 'rl4rs.pkl'))


In [9]:
rl4rs_results = []
r = RL4RS.load(os.path.join(pkl_path, 'rl4rs.pkl'))
r_train_loader, r_val_loader, r_test_loader, r_train_user_item_matrix, train_num_items = get_train_val_test_tmatrix_tnumitems(r, batch_size=20000)
len(r_train_loader), len(r)

(2, 45942)

In [10]:
for embeddings in ['neural','explicit', 'svd',  ]:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")

    model = NeuralClickModel(
        RecsysEmbedding(train_num_items, r_train_user_item_matrix, embeddings='neural'),
        output_dim=1
    ).to(device)

    _, metrics = train(model, 
       r_train_loader, r_val_loader, r_test_loader, device=device, lr=1e-3, num_epochs=5000, early_stopping=50,
       silent=True, 
    )
    
    metrics['embeddings'] = embeddings
    rl4rs_results.append(metrics)
    
pd.DataFrame(rl4rs_results).to_csv(f'results/rl4rs_{experiment_name}.csv')


Evaluating FlattenedGRU with neural embeddings


train:   0%|                                                                                                                 | 0/5000 [00:00<?, ?it/s]

Test before learning: {'f1': 0.5420801639556885, 'roc-auc': 0.5074925422668457, 'accuracy': 0.4934590756893158}


train... loss:1.3903907537460327:   0%|                                                                           | 1/5000 [00:36<50:11:26, 36.14s/it]

Test update: epoch: 0 |accuracy: 0.6443235278129578 | f1: 0.7836943864822388 | auc: 0.5275754928588867 | treshold: 0.01


...

train... loss:0.5928073823451996:  12%|████████▌                                                               | 596/5000 [1:28:35<7:00:37,  5.73s/it]

Test update: epoch: 595 |accuracy: 0.8501511216163635 | f1: 0.8838534355163574 | auc: 0.9223062992095947 | treshold: 0.61


train... loss:0.5957358777523041:  12%|████████▌                                                               | 597/5000 [1:28:41<7:01:32,  5.74s/it]

Test update: epoch: 596 |accuracy: 0.8510941863059998 | f1: 0.8844479322433472 | auc: 0.9224815368652344 | treshold: 0.62


train... loss:0.574576735496521:  12%|████████▋                                                               | 605/5000 [1:29:28<10:50:02,  8.87s/it]
